In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras import datasets, layers, models

df=pd.read_excel(r's&p_macrotrends.xlsx')
data = pd.DataFrame(df, columns= ['diff']).values
print(len(data))
plt.plot(data)
plt.show()


def create_time_steps(length):
        return list(range(-length, 0))


def show_plot(plot_data, delta, title):
        labels = ['History', 'True Future', 'Model Prediction']
        marker = ['.-', 'rx', 'go']
        time_steps = create_time_steps(plot_data[0].shape[0])
        if delta:
            future = delta
        else:
            future = 0

        plt.title(title)
        for i, x in enumerate(plot_data):
            if i:
                plt.plot(future, plot_data[i], marker[i], markersize=10,
                label=labels[i])
            else:
                plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
        plt.legend()
        plt.xlim([time_steps[0], (future+5)*2])
        plt.xlabel('Time-Step')
        return plt


def univariate_data(dataset, start_index, end_index, history_size, target_size):
            datas = []
            labels = []

            start_index = start_index + history_size
            if end_index is None:
                end_index = len(dataset) - target_size

            for i in range(start_index, end_index):
                indices = range(i-history_size, i)
    # Reshape data from (history_size,) to (history_size, 1)
                datas.append(np.reshape(dataset[indices], (history_size, 1)))
                labels.append(dataset[i+target_size])
            return np.array(datas), np.array(labels)
TRAIN_SPLIT = 18000
print('diocane')
tf.random.set_seed(13)
uni_train_mean = data[:TRAIN_SPLIT].mean()
uni_train_std = data[:TRAIN_SPLIT].std()
uni_data = (data - uni_train_mean) / uni_train_std
univariate_past_history = 10
univariate_future_target = 1

x_train_uni, y_train_uni = univariate_data(uni_data, 0, TRAIN_SPLIT,
                                             univariate_past_history,
                                             univariate_future_target)
x_val_uni, y_val_uni = univariate_data(uni_data, TRAIN_SPLIT, None,
                                         univariate_past_history,
                                         univariate_future_target)
BATCH_SIZE = 256
BUFFER_SIZE = 1000

train_univariate = tf.data.Dataset.from_tensor_slices((x_train_uni, y_train_uni))
train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
val_univariate = tf.data.Dataset.from_tensor_slices((x_val_uni, y_val_uni))
val_univariate = val_univariate.batch(BATCH_SIZE).repeat()
simple_lstm_model = tf.keras.models.Sequential([
      tf.keras.layers.LSTM(8, input_shape=x_train_uni.shape[-2:]),
      tf.keras.layers.Dense(1)
  ])

simple_lstm_model.compile(optimizer='adam', loss='mae')
EVALUATION_INTERVAL = 2000
EPOCHS = 10

simple_lstm_model.fit(train_univariate, epochs=EPOCHS,
                        steps_per_epoch=EVALUATION_INTERVAL,
                        validation_data=val_univariate, validation_steps=50)
for x, y in val_univariate.take(3):
    plotz = show_plot([x[0].numpy(), y[0].numpy(), simple_lstm_model.predict(x)[0]], 0,'diocane')
    plotz.show()

